# 공격수 데이터 분석 - Follower 제거 전

# 작업 데이터 받기

In [33]:
df_pos = pd.read_csv(r'C:\Users\Gk\Documents\dev\data\LinearRegression_Football_data\df_pos.csv', encoding='utf-8-sig', index_col=0)

# Position Rounding

In [34]:
df_pos.position = df_pos.position.round()
df_pos.position.unique()
df_atk = df_pos[df_pos.position == 4].append(df_pos[df_pos.position == 2])
df_atk.reset_index(drop=True)
df_ols = df_atk.drop(['player_name', 'goals_conceded', 'penalty_saved', 'position'], axis=1)

In [44]:
df_ols

,age,height,weight,rating,value,follower,shots_total,shots_on,goals_total,goals_assists,...,penalty_won,penalty_commited,penalty_success,penalty_missed,games_appearences,games_lineups,substitutes_in,substitutes_out,substitutes_bench,games_played
1,27.0,181.0,72.0,5.250000,32.0,740918,2.8168,1.0538,0.4432,0.0985,...,0.0394,0.0000,0.0985,0.0295,1.0538,0.9947,0.0591,0.0689,0.0689,101.5333
3,25.0,171.0,70.0,7.000000,32.0,652886,1.9736,0.8442,0.2282,0.2168,...,0.0114,0.0114,0.0000,0.0000,1.5515,0.9697,0.5818,0.6731,0.6160,87.6556
7,29.0,176.0,73.0,6.666667,96.0,30243196,2.6247,1.2356,0.5249,0.2261,...,0.0162,0.0000,0.0485,0.0000,1.1064,1.0418,0.0646,0.3634,0.0969,123.8222
12,29.0,177.0,73.0,6.777778,30.5,351898,3.3557,1.3951,0.5279,0.1760,...,0.0000,0.0000,0.1131,0.0251,1.1814,0.9677,0.2137,0.2262,0.2639,79.5667
13,29.0,175.0,73.0,5.750000,48.0,3141100,2.7988,1.1863,0.5183,0.2073,...,0.0230,0.0000,0.0346,0.0000,1.3015,1.0136,0.2879,0.4953,0.4262,86.8222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357,26.0,172.0,66.0,4.100000,16.0,170526,1.8926,0.4423,0.0617,0.1234,...,0.0000,0.0000,0.0103,0.0309,1.1211,1.0183,0.1029,0.2571,0.1337,97.2222
359,27.0,178.0,74.0,6.000000,16.0,820996,0.3110,0.0444,0.0444,0.0000,...,0.0000,0.0000,0.0000,0.0000,1.0365,1.0217,0.0148,0.1037,0.0296,67.5333
362,22.0,185.0,84.0,7.000000,20.0,152427,1.1419,0.3217,0.0643,0.0965,...,0.0161,0.0161,0.0000,0.0000,1.3831,0.9811,0.4021,0.3538,0.5147,62.1778
363,24.0,183.0,74.0,7.125000,36.0,529117,1.2701,0.2200,0.0500,0.0500,...,0.0100,0.0100,0.0000,0.0000,1.0701,1.0001,0.0700,0.1000,0.1000,99.9889


# OLS - Basis Model

In [87]:
dfX = df_ols.drop(['value'], axis=1)
dfy = df_ols['value']
df = pd.concat([dfX, dfy], axis=1)

formula = "value ~ scale(age) + \
            scale(goals_total) + \
            scale(goals_assists) + \
            scale(passes_total) + \
            scale(passes_key) + \
            scale(passes_accuracy) + \
            scale(dribbles_attempts) + \
            scale(dribbles_success) + \
            scale(penalty_won) + \
            scale(penalty_success) + \
            scale(games_appearences) + \
            scale(games_played)"

model = sm.OLS.from_formula(formula, data=df_ols)
result = model.fit()
print(result.summary())

##############################################################################
from sklearn.model_selection import KFold

scores = np.zeros(10)
cv = KFold(10, shuffle=True, random_state=0)
for i, (idx_train, idx_test) in enumerate(cv.split(df_ols)):
    df_train = df_ols.iloc[idx_train]
    df_test = df_ols.iloc[idx_test]
    
    model = sm.OLS.from_formula(formula, data=df_train)
    result_p = model.fit()
    
    pred = result_p.predict(df_test)
    rss = ((df_test.value - pred) ** 2).sum()
    tss = ((df_test.value - df_test.value.mean())** 2).sum()
    rsquared = 1 - rss / tss
    
    scores[i] = rsquared
    print("학습 R2 = {:.8f}, 검증 R2 = {:.8f}".format(result_p.rsquared, rsquared))
print("모델 성능 : {}".format(scores.mean()))

                            OLS Regression Results                            
Dep. Variable:                  value   R-squared:                       0.519
Model:                            OLS   Adj. R-squared:                  0.495
Method:                 Least Squares   F-statistic:                     21.60
Date:                Tue, 14 Jul 2020   Prob (F-statistic):           6.19e-32
Time:                        00:43:23   Log-Likelihood:                -1068.6
No. Observations:                 253   AIC:                             2163.
Df Residuals:                     240   BIC:                             2209.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept               

In [88]:
result.pvalues.sort_values(ascending=False)

scale(penalty_won)          1.458420e-02
scale(penalty_success)      1.148983e-02
scale(passes_total)         2.137890e-03
scale(dribbles_attempts)    1.870257e-03
scale(passes_key)           1.584247e-03
scale(games_appearences)    7.859117e-04
scale(passes_accuracy)      3.759480e-04
scale(dribbles_success)     2.078363e-04
scale(goals_assists)        5.465656e-05
scale(age)                  2.884805e-07
scale(games_played)         2.663664e-07
scale(goals_total)          3.312575e-10
Intercept                   9.351849e-95
dtype: float64

In [89]:
dfX = df_ols.drop(['value'], axis=1)
dfy = df_ols['value']
df = pd.concat([dfX, dfy], axis=1)

formula = "value ~ scale(age) + \
            scale(goals_total) + \
            scale(goals_assists) + \
            scale(passes_total) + \
            scale(passes_key) + \
            scale(passes_accuracy) + \
            scale(dribbles_attempts) + \
            scale(dribbles_success) + \
            scale(penalty_won) + \
            scale(penalty_success) + \
            scale(games_appearences) + \
            scale(games_played) + \
            scale(follower)"

model = sm.OLS.from_formula(formula, data=df_ols)
result = model.fit()
print(result.summary())

##############################################################################
from sklearn.model_selection import KFold

scores = np.zeros(10)
cv = KFold(10, shuffle=True, random_state=0)
for i, (idx_train, idx_test) in enumerate(cv.split(df_ols)):
    df_train = df_ols.iloc[idx_train]
    df_test = df_ols.iloc[idx_test]
    
    model = sm.OLS.from_formula(formula, data=df_train)
    result_p = model.fit()
    
    pred = result_p.predict(df_test)
    rss = ((df_test.value - pred) ** 2).sum()
    tss = ((df_test.value - df_test.value.mean())** 2).sum()
    rsquared = 1 - rss / tss
    
    scores[i] = rsquared
    print("학습 R2 = {:.8f}, 검증 R2 = {:.8f}".format(result_p.rsquared, rsquared))
print("모델 성능 : {}".format(scores.mean()))

                            OLS Regression Results                            
Dep. Variable:                  value   R-squared:                       0.541
Model:                            OLS   Adj. R-squared:                  0.516
Method:                 Least Squares   F-statistic:                     21.67
Date:                Tue, 14 Jul 2020   Prob (F-statistic):           1.47e-33
Time:                        00:43:56   Log-Likelihood:                -1062.8
No. Observations:                 253   AIC:                             2154.
Df Residuals:                     239   BIC:                             2203.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept               

# Feature들에 대해서 ANOVA 확인

In [55]:
model_full = sm.OLS.from_formula(
    "value ~ scale(age) + \
                scale(shots_on) + \
                scale(goals_total) + \
                scale(goals_assists) + \
                scale(passes_accuracy) + \
                scale(dribbles_attempts) + \
                scale(dribbles_success) + \
                scale(fouls_drawn) + \
                scale(cards_yellow) + \
                scale(penalty_won) + \
                scale(penalty_success) + \
                scale(games_appearences) + \
                scale(games_played) + \
                scale(follower)", data=df_ols)
model_reduced = sm.OLS.from_formula(
    "value ~ scale(age) + \
                scale(shots_on) + \
                scale(goals_total) + \
                scale(goals_assists) + \
                scale(passes_accuracy) + \
                scale(dribbles_attempts) + \
                scale(dribbles_success) + \
                scale(fouls_drawn) + \
                scale(cards_yellow) + \
                scale(penalty_won) + \
                scale(penalty_success) + \
                scale(games_appearences) + \
                scale(games_played)", data=df_ols)

sm.stats.anova_lm(model_reduced.fit(), model_full.fit())

C:\Users\Gk\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
C:\Users\Gk\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
C:\Users\Gk\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


,df_resid,ssr,df_diff,ss_diff,F,Pr(>F)
0,239.0,69996.876955,0.0,NaN,NaN,NaN
1,238.0,64620.287223,1.0,5376.589733,19.80227,0.000013


In [56]:
model_full = sm.OLS.from_formula(
    "value ~ scale(age) + \
                scale(shots_on) + \
                scale(goals_total) + \
                scale(goals_assists) + \
                scale(passes_accuracy) + \
                scale(dribbles_attempts) + \
                scale(dribbles_success) + \
                scale(fouls_drawn) + \
                scale(cards_yellow) + \
                scale(penalty_won) + \
                scale(penalty_success) + \
                scale(games_appearences) + \
                scale(games_played) + \
                scale(follower)", data=df_ols)

result = model_full.fit()
sm.stats.anova_lm(result, typ=2)

,sum_sq,df,F,PR(>F)
scale(age),8213.323543,1.0,30.250113,9.778616e-08
scale(shots_on),2632.990978,1.0,9.697448,2.071349e-03
scale(goals_total),8696.530975,1.0,32.029792,4.347585e-08
scale(goals_assists),4422.609018,1.0,16.288707,7.334183e-05
scale(passes_accuracy),8622.839592,1.0,31.758383,4.917546e-08
scale(dribbles_attempts),2415.148009,1.0,8.895120,3.156625e-03
scale(dribbles_success),3736.710301,1.0,13.762505,2.582367e-04
scale(fouls_drawn),1311.184541,1.0,4.829163,2.894626e-02
scale(cards_yellow),443.908535,1.0,1.634939,2.022667e-01
scale(penalty_won),3119.491869,1.0,11.489257,8.192536e-04
